In [1]:
import logging
import time
from datetime import datetime
from pathlib import Path

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
import dask.config
from distributed import LocalCluster, Client

In [ ]:
base_path = Path('/work/spd-scipy2020/')

In [3]:
dask.config.set({'temporary_directory': '/work/alewis/tmp'})
dask.config.config['distributed']['comm']['timeouts']['connect'] = '120s'
# dask.config.config['distributed']['scheduler']['allowed-failures'] = 12
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.3, 'spill': 0.5, 'pause': 0.7, 'terminate': 0.95}
# dask.config.config['distributed']['comm']['retry']['count'] = 5

In [4]:
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8787',
                       n_workers=6,
                       threads_per_worker=4,
                       memory_limit='3 GB')
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35243 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 24 Memory: 18.00 GB


In [5]:
df = dd.read_parquet(base_path / 'data/small_partitions_geohashed_gps_pts', engine='pyarrow')
# df = df.partitions[1217:].repartition(npartitions=8).persist()
df.head()

,latitude,longitude,geohash
index,,,
0,-90.0,-39.374215,50h000084212
1,-90.0,-5.624215,5bh000084212
2,-90.0,64.688241,j2n000081208
3,-90.0,90.000000,n00000000000
4,-90.0,98.438280,n0n0000840j8


In [ ]:
%%time 
t0 = time.time()
df2 = df.set_index('geohash', shuffle='disk').to_parquet(base_path / 'data/sorted_geohash_pts_distributed', engine='pyarrow', compression=None)

In [ ]:
dt_hr = (time.time() - t0)/60/60
with open(f'geohash_sort_time_distributed_{datetime.now()}.csv', 'w') as f:
    f.write(f'dt_hr,{dt_hr}')

In [8]:
# started at 4:20 pm